# Face Alignment

This notebook when run goes through every image in the specifed directories and using the MTCNN face detector detects and alignes faces.

In [ ]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from PIL import Image

# Initialise MTCNN detector
detector = MTCNN()

def align_face(image_path):
    """ Detects, aligns, and rotates face from an image using MTCNN. Overwrites the image if a face is found. """
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Skipping {image_path}, unable to read image.")
            return
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        original_size = (img.shape[1], img.shape[0]) 
        detections = detector.detect_faces(img_rgb)
        
        if detections:
            for detection in detections:
                if detection['confidence'] > 0.90:  
                    x, y, width, height = detection['box']
                    x, y = max(0, x), max(0, y) 
                    face = img_rgb[y:y+height, x:x+width]  
                    
                    # Get facial landmarks
                    keypoints = detection['keypoints']
                    left_eye = keypoints['left_eye']
                    right_eye = keypoints['right_eye']

                    # Convert to integers (fix TypeError)
                    left_eye = (int(left_eye[0]), int(left_eye[1]))
                    right_eye = (int(right_eye[0]), int(right_eye[1]))

                    # Compute angle to align the eyes
                    dx = right_eye[0] - left_eye[0]
                    dy = right_eye[1] - left_eye[1]
                    angle = np.degrees(np.arctan2(dy, dx))

                    # Rotate the image around the center of the eyes
                    center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
                    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
                    aligned_img = cv2.warpAffine(img_rgb, rot_matrix, (img.shape[1], img.shape[0]))

                    # Crop face again after rotation
                    aligned_face = aligned_img[y:y+height, x:x+width]

                    # Convert to PIL Image for resizing
                    face_pil = Image.fromarray(aligned_face)
                    face_pil = face_pil.resize(original_size) 
                    
                    # Save aligned face (overwrite original image)
                    face_pil.save(image_path)
                    print(f"Aligned and saved: {image_path}")
                    return 
        
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def process_directory(directory):
    """ Recursively processes all images in the given directory and its subdirectories. """
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                align_face(image_path)

In [ ]:
# Set input directory
input_dirs = ['AffectNet_Structured_Aligned', 'CK+_Structured_Aligned', 'FER_Structured_Aligned', 'RAF-DB_Structured_Aligned', 'JAFFE_Structured_Aligned']

for input_dir in input_dirs:
    if not os.path.exists(input_dir):
        print(f"Directory {input_dir} does not exist. Skipping.")
        continue
    print(f"Processing directory: {input_dir}")
    process_directory(input_dir)